In [1]:
!git clone https://github.com/derInformatiker/AIcrowd-AIBlitz7-Solution.git
!pip install -r AIcrowd-AIBlitz7-Solution/challenge2/requirements.txt
!pip install aicrowd-cli==0.1

Cloning into 'AIcrowd-AIBlitz7-Solution'...
remote: warning: multi-pack bitmap is missing required reverse index
remote: Enumerating objects: 305, done.
remote: Counting objects: 100% (305/305), done.
remote: Compressing objects: 100% (179/179), done.
remote: Total 305 (delta 177), reused 219 (delta 121), pack-reused 0
Receiving objects: 100% (305/305), 456.81 KiB | 24.04 MiB/s, done.
Resolving deltas: 100% (177/177), done.
     |████████████████████████████████| 10.1MB 18.8MB/s 
     |████████████████████████████████| 15.3MB 207kB/s 
     |████████████████████████████████| 28.2MB 168kB/s 
     |████████████████████████████████| 81kB 10.1MB/s 
     |████████████████████████████████| 11.5MB 47.4MB/s 
ERROR: tensorflow 2.4.1 has requirement numpy~=1.19.2, but you'll have numpy 1.20.2 which is incompatible.
ERROR: google-colab 1.0.0 has requirement pandas~=1.1.0; python_version >= "3.0", but you'll have pandas 1.0.5 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==

     |████████████████████████████████| 40kB 6.3MB/s 
     |████████████████████████████████| 163kB 24.5MB/s 
     |████████████████████████████████| 61kB 10.5MB/s 
     |████████████████████████████████| 204kB 53.2MB/s 
     |████████████████████████████████| 71kB 11.5MB/s 
     |████████████████████████████████| 51kB 9.0MB/s 
  Created wheel for aicrowd-cli: filename=aicrowd_cli-0.1.0-cp37-none-any.whl size=34929 sha256=745f5ecd9fadfc7b07c82093b1fd5c23da3d1359a34a2c14d5ce6035f31faf03
  Stored in directory: /root/.cache/pip/wheels/7e/ec/cc/a38391f23ad9bd044c91cd5e6e0a30f2a4265a1eb5adedf9f5
Successfully built aicrowd-cli


###RESTART RUNTIME TO USE NEW PACKAGES

In [1]:
API_KEY = ""  # Please enter your API Key from [https://www.aicrowd.com/participants/me]
!aicrowd login --api-key $API_KEY

API Key valid
Saved API Key successfully!


In [2]:
!aicrowd dataset download --challenge debris-detection

!rm -rf data
!mkdir data

!unzip -q train.zip  -d data/train
!unzip -q val.zip -d data/val
!unzip -q test.zip  -d data/test

!mv train.csv data/train.csv
!mv val.csv data/val.csv
!mv sample_submission.csv data/sample_submission.csv

sample_submission.csv: 100% 1.98M/1.98M [00:00<00:00, 4.18MB/s]
test.zip: 100% 259M/259M [00:33<00:00, 7.71MB/s]
train.csv: 100% 1.78M/1.78M [00:00<00:00, 3.67MB/s]
train.zip: 100% 1.04G/1.04G [02:11<00:00, 7.88MB/s]
val.csv: 100% 176k/176k [00:00<00:00, 764kB/s]
val.zip: 100% 104M/104M [00:18<00:00, 5.70MB/s]


In [3]:
import pandas as pd
import ast
import numpy as np
from detecto import core, utils, visualize
from matplotlib import pyplot as plt
from tqdm import tqdm
import cv2

In [4]:
data = pd.read_csv('data/train.csv')
fn, w, h, cl, boxes, imgid = [], [], [], [], [], []
counter = 0
for i in range(300):
    idx = int(data['ImageID'].iloc[i])
    bbox = ast.literal_eval(data['bboxes'].iloc[i])
    for box in bbox:
        fn.append(f'{idx}.jpg')
        w.append(512)
        h.append(512)
        cl.append('debris')
        boxes.append(box)
        imgid.append(counter)
        counter += 1
boxes = np.array(boxes).T
pd.DataFrame(
    {'filename':fn, 
     'width' : w, 
     'height' : h, 
     'class' : cl,
     'xmin' : boxes[0],
     'ymin' : boxes[2],
     'xmax' : boxes[1],
     'ymax':boxes[3],
     'image_id':imgid
    }
).to_csv('data/labels.csv',index = False)

In [6]:
dataset = core.Dataset('data/labels.csv','data/train')
model = core.Model(['debris'],pretrained = False)
m = model.fit(dataset,epochs=10, learning_rate=0.001, lr_step_size=5,verbose = True)

Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.cache/torch/hub/checkpoints/resnet50-19c8e357.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

  0%|          | 0/1137 [00:00<?, ?it/s]

Epoch 1 of 1
Begin iterating over training dataset


100%|██████████| 1137/1137 [03:56<00:00,  4.81it/s]


In [ ]:
out = []
for i in tqdm(range(5000)):
    image = utils.read_image(f'data/test/{i}.jpg')
    labels, boxes, scores = model.predict(image)
    bb = []
    for u,box in enumerate(boxes):
        box = box.cpu().tolist()
        bb.append([box[0],box[2],box[1],box[3],float(scores[u]+0.2 if scores[u]+0.2 <= 1 else 1)])
    out.append(bb)

In [ ]:
def draw_bboxes(img, bboxes, train,color=(255, 0, 0), thickness=1):
    for u, bbox in enumerate(bboxes):
        # if [x1, y1, x2, y2]
        img = cv2.rectangle(img, (int(bbox[0]),int(bbox[2])), (int(bbox[1]),int(bbox[3])), color, thickness)
    return img

In [ ]:
num = 5
i = utils.read_image(f'data/test/{num}.jpg')
#i = cv2.flip(i,0)
image = draw_bboxes(i,out[num],True)
#image = draw_bboxes(image,t[num],False,(0,255,0))
plt.clf()
plt.figure(figsize=(10,10))
plt.imshow(image)

In [ ]:
df = pd.read_csv('data/sample_submission.csv')
df.bboxes = out
df.to_csv('submission.csv',index = False)